In [1]:
import pyspark



In [2]:
spark = pyspark.sql.SparkSession.builder.appName("test").master("local[4]").config("spark.driver.memory", "4g").getOrCreate()

In [3]:

# Using forward slashes for the path
df = spark.read.csv("D:/STUFF/Projects/BigData_Project/data/all_reviews/all_reviews.csv", header=True, inferSchema=True)



In [4]:
df = df.drop('hidden_in_steam_china', 'steam_china_location')


In [14]:
# # Count total number of rows
# total_rows = df.count()
# print(f"Total number of rows: {total_rows}")

Total number of rows: 113885601


In [5]:
df.columns

['recommendationid',
 'appid',
 'game',
 'author_steamid',
 'author_num_games_owned',
 'author_num_reviews',
 'author_playtime_forever',
 'author_playtime_last_two_weeks',
 'author_playtime_at_review',
 'author_last_played',
 'language',
 'review',
 'timestamp_created',
 'timestamp_updated',
 'voted_up',
 'votes_up',
 'votes_funny',
 'weighted_vote_score',
 'comment_count',
 'steam_purchase',
 'received_for_free',
 'written_during_early_access']

In [6]:
# df.coalesce(1) \
#     .write \
#     .option("compression", "snappy") \
#     .parquet("D:/STUFF/Projects/BigData_Project/data/all_reviews/new")


In [7]:
# from pyspark.sql.functions import rand

# # 1. Load the Parquet file
# parquet_input_path = "D:/STUFF/Projects/BigData_Project/data/all_reviews/main_data.parquet"
# df_parquet = spark.read.parquet(parquet_input_path)

# # 2. Print its columns
# print(f"Columns in {parquet_input_path}:")
# print(df_parquet.columns)

# sample parquet generation

In [8]:

# # Load the parquet file
# df = spark.read.parquet("D:/STUFF/Projects/BigData_Project/data/all_reviews/main_data.parquet")


In [9]:

# # Print the total count to verify data was loaded
# total_count = 113885601

# # Sample 1 million records randomly
# # If the dataset has fewer than 1 million records, we'll take a fraction that would give us 1 million if we had enough
# sample_count = 1000000
# if total_count > sample_count:
#     sampled_df = df.sample(False, sample_count / total_count, seed=42)
# else:
#     # Take all records if we have fewer than 1 million
#     sampled_df = df


In [10]:
# sampled_df.head(5)

In [11]:

# # Verify the count of sampled records
# sampled_count = sampled_df.count()
# print(f"Number of sampled records: {sampled_count}")


In [12]:

# # Save the sampled records as a new parquet file
# sampled_df.write \
#     .option("compression", "snappy") \
#     .parquet("D:/STUFF/Projects/BigData_Project/data/all_reviews/sampled_1m_reviews")

# print("Sample parquet file created successfully.")

# # Stop Spark session


In [13]:
# sampled_df.coalesce(1) \
#     .write \
#     .option("compression", "snappy") \
#     .parquet("D:/STUFF/Projects/BigData_Project/data/all_reviews/sampled_1m_reviews")


# Sample CSV GENERATION

In [3]:
import pandas as pd
import numpy as np
import random
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, isnan, count, rand, trim, length
from pyspark.sql.types import IntegerType, LongType, BooleanType, FloatType

from pyspark.sql import SparkSession
import plotly.express as px
import pandas as pd

def get_top_10_games_by_reviews(parquet_dir):
    """
    Find the top 10 games with the most reviews from a directory of Parquet files,
    grouping by the 'game' column, and display results with a bar chart.
    
    Parameters:
    - parquet_dir: Path to the directory containing Parquet files
    """
    # Initialize Spark session
    spark = SparkSession.builder \
        .appName("TopGamesByReviews") \
        .config("spark.driver.memory", "8g") \
        .getOrCreate()
    
    try:
        # Load all Parquet files from the directory
        df = spark.read.parquet(parquet_dir)
        
        # Verify if 'game' column exists
        if "game" not in df.columns:
            spark.stop()
            raise ValueError("Column 'game' not found in the dataset. Available columns: " + ", ".join(df.columns))
        
        # Group by 'game', count reviews, and get top 10
        top_games = df.groupBy("game") \
                     .count() \
                     .orderBy(col("count").desc()) \
                     .limit(10) \
                     .toPandas()
        
        # Display results as a table
        print("Top 10 Games by Review Count:")
        display(top_games)  # Use display for better formatting in Jupyter
        
        # Create bar chart
        fig = px.bar(
            top_games,
            x="game",
            y="count",
            title="Top 10 Games by Review Count",
            labels={"game": "Game Name", "count": "Number of Reviews"},
            color="game",
            text="count"
        )
        fig.update_layout(xaxis_tickangle=45, showlegend=False)
        fig.show()
        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Stop Spark session
        spark.stop()

# Execute the query
parquet_dir = "D:/STUFF/Projects/BigData_Project/data/all_reviews/cleaned_reviews"
get_top_10_games_by_reviews(parquet_dir)

Error: An error occurred while calling o81.pandasStructHandlingMode. Trace:
py4j.Py4JException: Method pandasStructHandlingMode([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:321)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:329)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:842)


